# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
2.0599e18,"""../../../../data/SampleHDS.jso…","""4232270645518069064""","""5792800232930015063""","""11250248401649898808""","""17640332042434897908""",false
1.6468e19,"""../../../../data/SampleHDS.jso…","""16272240683797668166""","""8303227534869089985""","""11250248401649898808""","""13366465954096529673""",true
1.5511e19,"""../../../../data/SampleHDS.jso…","""7491389362649755568""","""6727341034999004767""","""16499033550806672045""","""17640332042434897908""",false
1.8214e19,"""../../../../data/SampleHDS.jso…","""4359734919332524166""","""8303227534869089985""","""10072823193125574191""","""17640332042434897908""",false
5.6863e18,"""../../../../data/SampleHDS.jso…","""16083163775507436635""","""5792800232930015063""","""10072823193125574191""","""9847606853545284503""",true
1.6962e18,"""../../../../data/SampleHDS.jso…","""15086565506382833972""","""6727341034999004767""","""7301344339681258079""",null,false
1.0871e18,"""../../../../data/SampleHDS.jso…","""12707964563275078413""","""6727341034999004767""","""10072823193125574191""","""9847606853545284503""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_City': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
3.4033e18,"""../../../../data/SampleHDS.jso…","""5845108970321347611""","""10798446440720164988""","""FirstMortgage30yr""","""13585951024685689590""","""Rejected"""
1.7200e19,"""../../../../data/SampleHDS.jso…","""2709010866277634428""","""16216396857943917597""","""FirstMortgage30yr""","""15773703285617732768""","""Accepted"""
2.3021e18,"""../../../../data/SampleHDS.jso…","""10097983815012181897""","""7447356215068475208""","""MoneyMarketSavingsAccount""","""13585951024685689590""","""Rejected"""
1.5218e19,"""../../../../data/SampleHDS.jso…","""9230082704161148163""","""16216396857943917597""","""BasicChecking""","""13585951024685689590""","""Rejected"""
6.2940e18,"""../../../../data/SampleHDS.jso…","""17520946990800869490""","""10798446440720164988""","""BasicChecking""","""2150278571940768666""","""Accepted"""
1.1980e19,"""../../../../data/SampleHDS.jso…","""17848082742653078957""","""7447356215068475208""","""UPlusFinPersonal""",null,"""Rejected"""
1.0936e19,"""../../../../data/SampleHDS.jso…","""1546748631741340933""","""7447356215068475208""","""BasicChecking""","""2150278571940768666""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
5.7984e18,"""../../../../data/SampleHDS.jso…","""13392123857515235014""","""15393642800912003828""","""FirstMortgage30yr""","""1830583726692876641""","""Rejected"""
6.0741e18,"""../../../../data/SampleHDS.jso…","""15608327712508599960""","""16572800884823360290""","""FirstMortgage30yr""","""11132467384543536292""","""Accepted"""
5.3186e18,"""../../../../data/SampleHDS.jso…","""2383345567591282693""","""13816584892836179087""","""MoneyMarketSavingsAccount""","""1830583726692876641""","""Rejected"""
2.5285e16,"""../../../../data/SampleHDS.jso…","""15738639893113182490""","""16572800884823360290""","""BasicChecking""","""1830583726692876641""","""Rejected"""
1.6077e18,"""../../../../data/SampleHDS.jso…","""12043268774758722510""","""15393642800912003828""","""BasicChecking""","""7367811498457351710""","""Accepted"""
3.7945e18,"""../../../../data/SampleHDS.jso…","""9495488715921065247""","""13816584892836179087""","""UPlusFinPersonal""",null,"""Rejected"""
3.1620e18,"""../../../../data/SampleHDS.jso…","""17853550980102570179""","""13816584892836179087""","""BasicChecking""","""7367811498457351710""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
filename=filename
Customer_City=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
9.8923e17,"""13702962371859555839""","""1979182253945390956""","""FirstMortgage30yr""","""12730983689357642336""","""Rejected"""
1.6179e18,"""8244837160827280908""","""9248074964152398957""","""FirstMortgage30yr""","""14907088036893333400""","""Accepted"""
7.3208e18,"""3612600793231765987""","""12884147541595207584""","""MoneyMarketSavingsAccount""","""12730983689357642336""","""Rejected"""
8.5442e17,"""5916219640005453391""","""9248074964152398957""","""BasicChecking""","""12730983689357642336""","""Rejected"""
1.0502e19,"""1224586219249808238""","""1979182253945390956""","""BasicChecking""","""10763309919626310878""","""Accepted"""
5.6877e18,"""16177322817218658305""","""12884147541595207584""","""UPlusFinPersonal""",null,"""Rejected"""
1.1564e19,"""1748094777817679968""","""12884147541595207584""","""BasicChecking""","""10763309919626310878""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'